# Object Detection Demo: Car Detection

This is a sample reference implementation to showcase Object detection (car in this case) with SSD and Async API.
Async API improves the overall frame-rate of the application by not waiting for the inference to complete but continue doing things ont he host while accelerator is busy. 
Specifically, this code demonstrates two parallel infer requests by processing the current frame while the next input frame is being captured. This essentially hides the latency of capturing.

## Overview of How it works?
At start-up the sample application reads the equivalent of command line arguments  and loads a network and image from the video input to the Inference Engine (IE) plugin. 
A job is submitted to the hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Core CPU, Intel® Movidius™ and/or Neural Compute Stick)
After the inference is completed, the output videos are appropriately stored in the /results directory which can then be viewed within the Jupyter Notebook instance

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed actual Edge hardware
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import some dependencies needed for displaying the results in this notebook
(tip: use **crtl+enter** to run the cell but stay on the same cell)

In [ ]:
from demoutils import videoHTML,liveQstat

### 0.2  (Optional-step): Original video without inference

If the user is curious to see the input video, run the following cell to view the orignal video used for inference and object detection.

In [ ]:
!ln -s /data/reference-sample-data/object-detection-python/cars_1900.mp4 
videoHTML('cars video', 'cars_1900.mp4')

## Step 1 : Using OpenVINO

The complete listing of source code for this example is in [object_detection_demo_ssd_async.py](object_detection_demo_ssd_async.py).

Let's follow the main aspects of this code to see how OpenVINO works.
The application reads equivalent of command line arguments and loads a network to the Inference Engine, After getting a frame from the OpenCV's VideoCapture API, it is ready to perform Inference.

**Command line arguments options and how they are interpreted in the application source code**

```
python3 object_detection_demo_ssd_async.py -m mobilenet-ssd/$3/mobilenet-ssd.xml 
                                           -i video/cars_1900.mp4
                                           -o $1
                                           -d $2
                                           -l extension/libcpu_extension.so

```

##### The description of the arguments used in the argument parser is the command line executable equivalent.
* -m location of the **mobilenet-ssd** pre-trained model which has been pre-processed using the **model optimzer**
   There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware
   (**Note** we are using mobilenet-ssd in this example. However, OpenVINO's Inference Engine is compatible with other neural network architectures such as AlexNet*, GoogleNet*, MxNet* etc.,)    

* -i location of the input video stream (video/cars_1900.mp4)
* -o location where the output file with inference needs to be stored. (results/core or results/xeon or results/gpu)
* -d Type of Hardware Acceleration (CPU or GPU or MYRIAD)
* -l Absolute path to the shared library and is currently optimized for core/xeon (extension/libcpu_extension.so )


### 1.1 Choosing Device

First, we must select the device used for the inferencing. This is done by loading the appropriate plugin to initialize the specified device and load the extensions library (if specified) provided in the extension/ folder for the device.


The plugin class is **`IEPlugin`** and can be constructed as follows:

```python
# Parsing arguments to the python script. Code for constructing argparser this is not shown here.
args = argparser.parse_args()

# Plugin initialization for specified device. 'plugin_dirs' is an optional argument.
plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)

# Loading any additional exension libraries
if args.cpu_extension and 'CPU' in args.device:
    plugin.add_cpu_extension(args.cpu_extension)
```
**Note**
Currently, three types of plugins are supported: CPU, GPU,  and MYRIAD
CPU plugin may require additional extensions to improve performance.
Use `add_cpu_extension` function to load these additional extensions.


### 1.2 Read the IR (Intermediate Representation) model

Intel Model Optimizer creates Intermediate Representation (IR) models that are optimized for different Intel hardware.
We can import these optimized models (weights) into our neural network using **`IENetwork`**. 
```python
# Importing network weights from IR models.
net = IENetwork.from_ir(model=model_xml, weights=model_bin)

# Some layers in IR models may be unsupported by some plugins. 
if "CPU" in plugin.device:
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters "
                  "using -l or --cpu_extension command line argument")
        sys.exit(1)
```

### 1.3 Load the network on to the model

Once we have the plugin and the network, we can load the network into the plugin using **`plugin.load`**.

```python
# Loading IR model to the plugin.
exec_net = plugin.load(network=net, num_requests=2)

# Read and pre-process the input image/video.
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob]
```

```python
# Determine the source of video, we will use pre-recorded input video file in this example, but it can be modified to use a camera with input argument 'cam'
if args.input == 'cam':
        input_stream = 0
        out_file_name = 'cam'
    else:
        input_stream = args.input
```

### 1.4 Start video capture using OpenCV 

Now we are ready to capture the frames from the video sample using **OpenCV VideoCapture** API.
Upon getting the frame we are ready to perform inference.

```python
cap = cv2.VideoCapture(input_stream)
```

## Step 2: Run the workload on the IoT DevCloud

All the code up to this point is run within the notebook instance running on the Developer machine. Now we are ready run the inference which can be executed on an actual Edge Machine by creating a job to be submitted on a queue to run on the first available Edge hardware that matches the user choice.

### 2.1 Creating job file
The job file is written in bash, and will be executed directly on the compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "object_detection_job.sh"



In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $1

# Running the object detection code
python3 object_detection_demo_ssd_async.py -m /data/reference-sample-data/models/mobilenet-ssd/$3/mobilenet-ssd.xml \
                                           -i /data/reference-sample-data/object-detection-python/cars_1900.mp4 \
                                           -o $1 \
                                           -d $2 \
                                           -l /data/reference-sample-data/extension/libcpu_extension.so

### 2.2 Understand how job queue is submitted

Now that we have the job script, we can submit the jobs to compute nodes using the `qsub` command.
We can submit object_detection_job to 4 different types of nodes simultaneously or just one node at at time.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets use name the job so that it is easier to distinguish between them.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [ ]:
!pbsnodes | grep properties | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes.

### 2.3 Job queue submission

Each of the next 4 cells below will submit jobs to different nodes Core/Xeon/GPU or Myriad .
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

**Note** You can submit all 4 jobs at once or follow one at a time. 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 
(tip: **shift+enter or control+enter** will run the cell and let you go to the next. So you can hit **shift+enter or control+enter** multiple times to quickly run multiple cells)

#### submitting to a node with Intel Core CPU

In [ ]:
!qsub object_detection_job.sh -l nodes=1:iei-tank-core -F "results/core CPU FP32" -N obj_det_core

#### submitting to a node with Intel Xeon CPU

In [ ]:
!qsub object_detection_job.sh -l nodes=1:iei-tank-xeon -F "results/xeon CPU FP32" -N obj_det_xeon

#### submitting to a node with Intel Core CPU and using the onboard Intel GPU

In [ ]:
!qsub object_detection_job.sh -l nodes=1:iei-tank-core -F "results/gpu GPU FP32" -N obj_det_gpu

#### submitting to a node with Intel Movidius NCS (Neural Computing Stick)

In [ ]:
!qsub object_detection_job.sh -l nodes=1:iei-tank-movidius -F "results/myriad MYRIAD FP16" -N obj_det_myriad

### 2.4 Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job: this is your current jupyter notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.
**Note** currently it takes a anywhere from 3-5 minutes depending on the number of users accessing the edge nodes.

## Step 4: View Results

Once the jobs are completed, the stdout and stderr output stores in files with names of the form (based on our `-N` option):

`obj_det_{type}.o{JobID}`

`obj_det_{type}.e{JobID}`

But for this script, the main output is the mp4 videos which are stored in the `results/` directory.
We wrote a short utility script that will display these videos with in the notebook.
See `demoutils.py` if interested in understanding further how the results are displayed in notebook.


In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 'results/core/cars_1900.mp4', 'results/core/stats.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)','results/xeon/cars_1900.mp4','results/xeon/stats.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 'results/gpu/cars_1900.mp4','results/gpu/stats.txt')

In [ ]:
videoHTML('IEI Tank + Myriad (Intel Core + Movidius)','results/myriad/cars_1900.mp4','results/myriad/stats.txt')